In [74]:
import pandas as pd
from pandas import DataFrame
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime

In [75]:
pmjsonfile = open('/Users/raju.pillai/mydata/DAdata/DataAcquisition/output/json/ihg/IHG_property.json','r')
chainfile = open('/Users/raju.pillai/mydata/DAdata/DataAcquisition/source/ihg/dimensions/chain.txt','r')
brandfile = open('/Users/raju.pillai/mydata/DAdata/DataAcquisition/source/ihg/dimensions/brand.txt','r')

In [76]:
pmjsondf = DataFrame(pd.read_json(pmjsonfile))
chaindata = DataFrame(pd.read_csv(chainfile, sep='|', engine='python'))
branddata = DataFrame(pd.read_csv(brandfile, sep='|', engine='python'))

In [77]:
chain_brand_combined_df = pd.merge(chaindata, branddata, left_on='chain_code', right_on='chain_code', how='inner')

In [78]:
property_chain_brand_combined_df = pd.merge(chain_brand_combined_df, pmjsondf, left_on='brand', right_on='property_brand', how='inner' )

In [79]:
now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [80]:
property_chain_brand_combined_df['file_source'] = 'IHG_property.json'
property_chain_brand_combined_df['file_timestamp'] = now

In [81]:
osfdata = (property_chain_brand_combined_df[[
  'chain_code'
, 'chain_name'
, 'property_brand'
, 'brand_name'
, 'property_id'
, 'property_name'
, 'property_addr1'
, 'property_addr2'
, 'property_city'
, 'property_state'
, 'property_country'
, 'property_postal'
, 'property_phone'
, 'property_fax'
, 'property_email'
, 'property_tax_id'
, 'property_fund_curr'
, 'property_invoice_currency'
, 'property_delivery_method'
, 'property_direct_debit'
, 'property_location'
, 'file_source'
, 'file_timestamp'
]]).astype(str)

In [82]:
osfdata.columns

Index(['chain_code', 'chain_name', 'property_brand', 'brand_name',
       'property_id', 'property_name', 'property_addr1', 'property_addr2',
       'property_city', 'property_state', 'property_country',
       'property_postal', 'property_phone', 'property_fax', 'property_email',
       'property_tax_id', 'property_fund_curr', 'property_invoice_currency',
       'property_delivery_method', 'property_direct_debit',
       'property_location', 'file_source', 'file_timestamp'],
      dtype='object')

In [83]:
table = pa.Table.from_pandas(osfdata)

In [84]:
 pq.write_to_dataset(table, root_path = '/Users/raju.pillai/mydata/DAdata/DataAcquisition/output/pqt/ihg/property_master_flat',partition_cols=['chain_code','file_timestamp'], compression='none', use_dictionary=False)

In [26]:
#df = pd.read_parquet('/Users/raju.pillai/mydata/DAdata/DataAcquisition/output/pqt/ihg/property_master')

In [28]:
#df["chain_id"].value_counts()